### Tarea 4: Selección de características

La idea de este apartado es utilizar nuestros datos para nuevamente hacer una anlisis en este caso me gustaria usar la tecnica de componentes principales para agrupar las caracteristicas que pueden tener los jugadores.

In [1]:
import pandas as pd
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn.preprocessing import StandardScaler

M_Final_DB = pd.read_csv('C:/Users/Mauricio/Documents/Maestría en Ciencia de Datos/Segundo Tetramestre/Aprendizaje Automatico/Final_DB_Modificado.csv')

X = M_Final_DB.drop(['WS','Player','G',"Salary"],axis = 1)
X_num = X.select_dtypes(include='number')
y = M_Final_DB["WS"]

X_scaled = StandardScaler().fit_transform(X_num)

selector = SelectKBest(score_func=mutual_info_regression, k=10)

X_selected = selector.fit_transform(X_scaled, y)

selected_features = X_num.columns[selector.get_support()]
print("Características seleccionadas por SelectKBest:")
print(selected_features)

Características seleccionadas por SelectKBest:
Index(['Unnamed: 0', 'BPM', 'VORP', 'PER', 'Age', '2P%', '3P%', 'FTA', 'PTS',
       'Reliability'],
      dtype='object')


Dado que nuestros datos eran no parametricos tuvimos que usar **mutual_info_regression** lo que me parece interesante de esta prueba es que las variables relacionadas con las WS no incluye ninguna variable defensiva aun y cuando el calculo de este si lo hace, lo que me suena interesante y podria ser razón para descartar el modelo.

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

pca = PCA()
X_pca = pca.fit_transform(X_scaled)

eigenvalores = pca.explained_variance_
varianza_explicada = pca.explained_variance_ratio_
tabla_pca = pd.DataFrame({
    "Componente": [f"PC{i+1}" for i in range(len(eigenvalores))],
    "Eigenvalor": np.round(eigenvalores, 3),
    "Varianza Explicada (%)": np.round(varianza_explicada * 100, 2),
    "Varianza Acumulada (%)": np.round(np.cumsum(varianza_explicada) * 100, 2)
})

print(tabla_pca)

   Componente  Eigenvalor  Varianza Explicada (%)  Varianza Acumulada (%)
0         PC1       6.172                   29.32                   29.32
1         PC2       4.109                   19.52                   48.85
2         PC3       2.506                   11.91                   60.75
3         PC4       1.429                    6.79                   67.54
4         PC5       1.066                    5.07                   72.61
5         PC6       1.042                    4.95                   77.56
6         PC7       0.842                    4.00                   81.56
7         PC8       0.668                    3.17                   84.73
8         PC9       0.626                    2.98                   87.71
9        PC10       0.503                    2.39                   90.10
10       PC11       0.463                    2.20                   92.30
11       PC12       0.423                    2.01                   94.31
12       PC13       0.330             

In [3]:
loadings = pd.DataFrame(pca.components_.T, columns=[f"PC{i+1}" for i in range(len(X_num.columns))], index=X_num.columns)
print("Cargas de variables por componente:")
print(loadings.round(3))

Cargas de variables por componente:
               PC1    PC2    PC3    PC4    PC5    PC6    PC7    PC8    PC9  \
Unnamed: 0  -0.249 -0.060  0.426  0.046  0.178  0.129 -0.013  0.009  0.136   
USG%         0.325 -0.159  0.266  0.118  0.069 -0.030 -0.052  0.036 -0.056   
BPM          0.324  0.063 -0.273 -0.127  0.231  0.074  0.051  0.063  0.092   
VORP         0.327  0.009 -0.209 -0.150  0.025  0.083  0.036  0.218 -0.010   
PER          0.369  0.132 -0.074  0.044  0.143  0.056  0.076 -0.070  0.078   
Age          0.040 -0.047 -0.185 -0.120 -0.236  0.821 -0.206 -0.148 -0.038   
2PA          0.314  0.099  0.276  0.046 -0.137 -0.040  0.101 -0.084  0.075   
2P%          0.113  0.270 -0.249  0.196  0.076 -0.116 -0.067 -0.388  0.521   
3PA          0.032 -0.383 -0.048  0.198  0.351  0.006 -0.166  0.190 -0.173   
3P%          0.022 -0.285 -0.250 -0.033  0.370 -0.044 -0.348  0.041  0.200   
FTA          0.311  0.030  0.256  0.093 -0.077  0.007  0.163 -0.124  0.005   
FT%          0.109 -0.284 -0

Con estas tablas respectivamente podemos concluir en el primer caso que idealemnte usar 4 Componentes seria lo ideal pues ya con esto es suficiente para cubrir el 70% de la variabilidad de los datos. Pero con la finalidad de poder obtener más diferencias en los estilos de juego usaremos 6 componentes.

A su vez podemos usar la segunda tabla los loadings, para interpretar que variables tienen mayor perso en estos componentes y con esto podemos definir estilos de juego:

- En el caso de CP1 tenemos que sus variables con más peso nos denotan que muestra las caracteristicas de jugadores con alto nivel de producción en puntos que demás esta acompañado por alta eficiencia.

- En el caso de CP2 tenemos que sus variables denotan a rebotadores y defensores de la pintura que mal juego exterior especificamente tiro perimetral.

- En CP3 podemos ver jugadores contrarios al caso uno, jugadores muy poco eficientes y consistencia en el juego

- En CP4 podemos ver jugadores que son tiradores de larga distancia que tienen una mala defensa perimetral y pasn poco la bola

- CP5 captura la veterania, juagadores que ya son más grandes y denota las caracteristicas que suelen perder por la edad.

- Por ultimo CP6 captura jugadores con un muy buen aporte defensivo pero que no sulen jugar mucho, probablemnte jugadores que suelen lesionarse frecuentemente.